## Part 1: Preprocessing

In [47]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [48]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [49]:
# Create y_df with the Attrition and Department columns
y_attrition = attrition_df[['Attrition']]
y_department = attrition_df[['Department']]









In [50]:
# Create a list of at least 10 column names to use as X data
X_data_list = ["Education",
              "Age",
              "DistanceFromHome",
              "JobSatisfaction",
              "OverTime",
              "StockOptionLevel",
              "WorkLifeBalance",
              "YearsAtCompany",
              "YearsSinceLastPromotion",
              "NumCompaniesWorked"
              ]

# Create X_df using your selected columns

X = attrition_df[X_data_list]

# Display the first few rows of X_df
X.head

# Show the data types for X_df
X.dtypes



Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [51]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_attrition_train, y_attrition_test, y_department_train, y_department_test  = train_test_split(X, y_attrition, y_department, random_state=1)





In [52]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train['OverTime'] = X_train['OverTime'].map({'Yes': 1, 'No': 0}).astype('int64')
X_test['OverTime'] = X_test['OverTime'].map({'Yes': 1, 'No': 0}).astype('int64')

















In [53]:
X_train.dtypes

Education                  int64
Age                        int64
DistanceFromHome           int64
JobSatisfaction            int64
OverTime                   int64
StockOptionLevel           int64
WorkLifeBalance            int64
YearsAtCompany             int64
YearsSinceLastPromotion    int64
NumCompaniesWorked         int64
dtype: object

In [54]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler = StandardScaler().fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [55]:
# Create a OneHotEncoder for the Department column
enc = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
enc.fit(y_department_train)

# Create two new variables by applying the encoder
# to the training and testing data
y_department_train_enc = enc.transform(y_department_train)
y_department_test_enc = enc.transform(y_department_test)


In [56]:
y_department_train_enc

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [57]:
# Create a OneHotEncoder for the Attrition column
enc = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
enc.fit(y_attrition_train)

# Create two new variables by applying the encoder
# to the training and testing data
y_attrition_train_enc = enc.transform(y_attrition_train)
y_attrition_test_enc = enc.transform(y_attrition_test)


In [58]:
y_attrition_train_enc

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

## Create, Compile, and Train the Model

In [59]:
# Find the number of columns in the X training data
len(X_train.columns)


10

In [60]:
# Create the input layer
input_layer = layers.Input(shape=(X.shape[1],), name='input_features')

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(128, activation='relu')(shared_layer1)

In [61]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(32, activation='relu')(shared_layer2)

# Create the output layer
department_output = layers.Dense(3, activation='softmax', name='department_output')(department_hidden)



In [62]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(32, activation='relu')(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(2, activation='sigmoid', name='attrition_output')(attrition_hidden)


In [63]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy',
                    'attrition_output': 'binary_crossentropy'},
              metrics={'department_output': 'accuracy',
                       'attrition_output': 'accuracy'})

# Summarize the model
model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_features      │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 64)        │        704 │ input_features[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 128)       │      8,320 │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 32)        │      4,128 │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 32)        │      4,128 │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         99 │ dense_10[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         66 │ dense_11[0][0]    │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [64]:
y_attrition_train.shape

(1102, 1)

In [65]:
y_attrition_train

,Attrition
464,No
512,No
480,Yes
680,No
1023,No
...,...
715,No
905,No
1096,No
235,No


In [66]:
y_department_train

,Department
464,Research & Development
512,Research & Development
480,Sales
680,Research & Development
1023,Research & Development
...,...
715,Research & Development
905,Research & Development
1096,Human Resources
235,Sales


In [67]:
y_department_train.shape

(1102, 1)

In [68]:
# Train the model
model.fit(
    X_train,
    {'department_output': y_department_train_enc, 'attrition_output': y_attrition_train_enc},
    epochs=100,
    batch_size=32,
    validation_split=0.2
)



Epoch 1/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - attrition_output_accuracy: 0.8643 - attrition_output_loss: 0.4546 - department_output_accuracy: 0.6315 - department_output_loss: 0.8952 - loss: 1.3500 - val_attrition_output_accuracy: 0.8281 - val_attrition_output_loss: 0.4706 - val_department_output_accuracy: 0.6109 - val_department_output_loss: 0.8687 - val_loss: 1.3399
Epoch 2/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.8431 - attrition_output_loss: 0.4144 - department_output_accuracy: 0.6300 - department_output_loss: 0.8436 - loss: 1.2578 - val_attrition_output_accuracy: 0.8281 - val_attrition_output_loss: 0.4681 - val_department_output_accuracy: 0.6109 - val_department_output_loss: 0.8443 - val_loss: 1.3124
Epoch 3/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.8575 - attrition_output_loss: 0.3744 - department_output_accuracy: 0.6657 - department_output_loss: 0.7814 - loss: 1.1556 - val_attrition_output_accuracy: 0.8281 - 

In [69]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test, {'department_output': y_department_test_enc, 'attrition_output': y_attrition_test_enc})
test_results


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.8040 - attrition_output_loss: 0.6272 - department_output_accuracy: 0.5958 - department_output_loss: 1.0165 - loss: 1.6497


[1.5283340215682983,
 0.9424486756324768,
 0.5467122793197632,
 0.823369562625885,
 0.6195651888847351]

# Print the accuracy for both department and attrition
Department Predictions Accuracy: 0.6122

Attrition Predictions Accuracy: 0.7974




# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Yes - there are other choices, such as precision, when minimizing false positives is a priority, and recall, when it is important to capure relevant cases, but neither of them seems well suited for this data. The data is not unbalanced, which makes it ideal for the accuracy metric. F1, which balances precision and recall, could be a good choice.

2. Softmax for the department data and sigmoid for the attrition data. Sigmoid is used for binary clasification, such as whether an employee is likely or not to leave. Softmax is used for multiclass questions, such as predicting the department that best fits an employee.   

3. Use of the F1 metric, which balances precision and recall, could be a good choice to try with this data. We could add additional column names to the X data. And it could be good to have more data.